In [1]:
%load_ext autoreload

import os, sys
sys.path.insert(0, "../vimtm/")
sys.path.insert(0, "../../miRExplore/python/")

%autoreload 2
from ViralMirnaExtraction import *


/home/users/joppich/.local/lib/python3.6/site-packages/pdfminer/pdfdocument.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend


In [2]:
def find_relevant_files(infolders):

    if not isinstance(infolders, list):
        infolders = [infolders]

    relFiles = []
    for folder in infolders:
        for path, subdirs, files in os.walk(folder):
            for name in files:
                if name.upper().endswith(("XML", "XLSX", "DOCX", "PDF", "ZIP")):
                    relFiles.append( os.path.join(path, name) )

    return relFiles

all_relevant_files = find_relevant_files(["./", "../covidtexts/"])
all_relevant_files


['./test.pdf',
 './PMC8733928/PMC8733928_DataSheet_1.zip',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File1.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File2.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File3.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File4.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File5.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File6.pdf',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File7.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/767726_SupMaterial/Supplementary File8.xlsx',
 './PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._Cove

In [4]:
sdf, identified_mirnas = process_files([x for x in reversed(all_relevant_files)])

#sdfSmall, imSmall = process_files(['./PMC7381279/PMC7381279_Data_Sheet_3.XLSX'])




../covidtexts/PMC8294073.xml
Sheets ['Table0.0']
Testing FASTA
Num FASTA miRNAs 0
Testing COLUMNS
Table0.0 (35, 6)
set mirName 0 None Name
set mirSeq 0 5 5′ mature Sequence
Name: 0 Name
Seq:  5 5′ mature Sequence
Reason:  SEQ_COLUMN
5 0 SEQ_COLUMN
Num COLUMNS miRNAs 34



../covidtexts/PMC7382400.xml
Sheets ['Table0.0', 'Table1.0', 'Table2.0', 'Table3.0']
Testing FASTA
Num FASTA miRNAs 0
Testing COLUMNS
Table0.0 (14, 8)
set mirName 1 None Predicted pre-miRNAs
set mirName 7 None pre-miRNAs Sequence
[]
Table1.0 (14, 3)
set mirName 0 None Predicted pre-miRNA
[]
Table2.0 (27, 4)
set mirName 0 None Mature miRNAs
set mirSeq 0 3 Mature miRNAs sequence
Name: 0 Mature miRNAs
Seq:  3 Mature miRNAs sequence
Reason:  SEQ_COLUMN
3 0 SEQ_COLUMN
Table3.0 (3, 5)
set mirName 0 None Mature miRNAs
[]
Num COLUMNS miRNAs 26



../covidtexts/PMC7278893.xml
Sheets ['Table0.0', 'Table1.0']
Testing FASTA
Num FASTA miRNAs 0
Testing COLUMNS
Table0.0 (19, 3)
Name: 0 miRNA
Seq:  0 miRNA
Reason:  NAME_SEQ_COLUMN

ERROR:root:Error loading file XML: ./PMC7395633/PMC7395633_mmc1/Data Profile.xml


Testing FASTA
Num FASTA miRNAs 0
Testing COLUMNS
1 (45, 7)
set mirSeq None 3 Sequences (5’ ‐> 3’)
[]
3 (51, 7)
set mirName 2 None v‐miRNAs
set mirName 0 None V‐miRNA1
set mirSeq 0 3 Size  Sequence 5’‐>3’
Name: 0 V‐miRNA1
Seq:  3 Size  Sequence 5’‐>3’
Reason:  SEQ_COLUMN
3 0 SEQ_COLUMN
4 (15, 6)
[]
2 (10, 5)
[]
5 (16, 2)
[]
Num COLUMNS miRNAs 21



./PMC8307234/PMC8307234_cells-10-01762-s001.zip



./PMC7395633/PMC7395633_mmc1/Data Profile.xml



./PMC7395633/PMC7395633_mmc2.docx
set mirName 0 None SARS-CoV2 miRNA
set mirName 1 None Human miRNA
[]
set mirName 0 None miRNA (SARS-CoV2)
set mirName 1 None miRNA (Human)
[]



./PMC7395633/PMC7395633_mmc1.zip



./PMC7381279/PMC7381279_Data_Sheet_6.DOCX
set mirName 3 None Our predicted miRNA
set mirName 3 None 5'stem-miRNA 56
set mirName 3 None 5'stem-miRNA 56
set mirName 3 None 5'stem-miRNA 437
set mirName 3 None 3'stem-miRNA 251
[]



./PMC7381279/PMC7381279_Data_Sheet_5.XLSX
Sheets ['SARS-CoV', 'SARS-CoV-2']
Testing FASTA
Num FASTA miRNAs

Testing FASTA
Num FASTA miRNAs 0
Testing COLUMNS
8 (1, 4)
[]
9 (3, 4)
[]
10 (8, 12)
[]
11 (46, 7)
[]
12 (52, 6)
[]
13 (42, 6)
[]
14 (65, 11)
[]
15 (21, 8)
[]
2 (77, 7)
[]
3 (79, 3)
[]
4 (34, 5)
[]
5 (13, 8)
[]
6 (29, 12)
[]
7 (9, 10)
[]
Num COLUMNS miRNAs 0



./PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._~$Supplementary File3.xlsx



./PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._~$Supplementary File1.xlsx



./PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._~$RS-C...le gene(1)-1126.docx



./PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._WB original plot.ZIP



./PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._Supplementary File8.xlsx



./PMC8733928/PMC8733928_DataSheet_1/767726_SupMaterial(1)/__MACOSX/767726_SupMaterial/._Supplementary File7.xlsx



./PMC8733928/PMC8733928_DataSheet_1/76

In [5]:
seq2mirnas = mirnaseq2mirnas(identified_mirnas)

NameError: name 'mirnaseq2mirnas' is not defined

In [ ]:
for x in seq2mirnas:
    print(x, seq2mirnas[x])

In [ ]:
prev = None
for x in set([x for x in identified_mirnas]):
    if x.mirna_id == "SARS-CoV-2-miR-080":
        print(x, x.mirna_seq, x.__eq__(prev), hash(x))
        prev = x

In [ ]:
mirnaSet = DocumentMirnaSequenceSet(identified_mirnas)

In [ ]:
mirnaSet.run_blast(humanXML="mirna_seqs.aligned.human.tsv", virusXML="mirna_seqs.aligned.virus.tsv", try_to_load=False)

In [ ]:
mirnaSet.assign_blast_matches()

In [ ]:
mirnasByOrigin = mirnaSet.get_mirna_by_origin()

In [ ]:
mirnasByOrigin["UNKNOWN"]

In [ ]:
for origin in mirnasByOrigin:
    for x in mirnasByOrigin[origin]:
        if len(x.bestmatches["virus"]) > 0:
            print(x)
            print(x.mirna_id)
            print(x.bestmatches)
            print(x.bestmatches["virus"])
            #print(x.bestmatches["virus"]["allhits"])
            
        break
    break


In [ ]:
mirnaSet.blast_virus[mirnaSet.blast_virus["query acc.ver"] == "MR109"]

In [ ]:
[x for x in mirnasByOrigin]

In [ ]:
len(mirnasByOrigin)

In [ ]:
for x in sorted(mirnasByOrigin):
    print(x, len(mirnasByOrigin[x]))

In [ ]:
mirnasByOrigin["UNKNOWN"]

In [ ]:
mirnaUpsetPlot(seq2mirnas)

In [ ]:
vaDB = ViralAccessionDB()
vaDB.annotate_virus_seq_name(mirnasByOrigin)
vaDB.annotate_matching_taxids(mirnasByOrigin)

In [ ]:
for x in sorted(mirnasByOrigin):
    print(x, vaDB.doc2tax.get(x, "-"))

In [ ]:
origin2tax2mirna, origin2tax2mirnaseq, tax2mirnas, tax2mirnaseq, _ = mirnaset.get_mirna_by_origin_taxid()

for taxID in tax2mirnas:
    print(taxID, len(tax2mirnas[taxID]), len(tax2mirnaseq[taxID]))

In [ ]:
for docID in sorted(origin2tax2mirna):
    for taxID in origin2tax2mirna[docID]:
        print(docID, taxID, len(origin2tax2mirna[docID][taxID]), len(origin2tax2mirnaseq[docID][taxID]))

In [ ]:
df = mirnaset.print_mirnas_table()
df.to_csv("vimtm_mirna.df", sep="\t", index=False)

display(df)

In [ ]:
df.METHOD.value_counts().plot(kind='bar')

In [ ]:
list(allmirnas)[25].__dict__

In [ ]:
author2pmc = {
"demirci": "PMC7278893",
"khan": "PMC7381279",
"saini": "PMC7382400",
"sarma": "PMC7395633",
"merino": "PMC7717134",
"aydemir": "PMC7773562",
"yu": "PMC7834301",
"roy": "PMC8078050",
"liu": "PMC8193712",
"fu": "PMC8257610",
"rahaman": "PMC8294073",
"meng": "PMC8307234",
"karimi": "PMC8358877",
"pawlica": "PMC8719879",
"zhu": "PMC8733928",
"singh": "PMC8811647"
}

In [ ]:
df = pd.read_csv("mirna_with_authors.tsv", sep="\t", encoding = 'unicode_escape', engine ='python', header=None)
df.columns = ["mirname", "mirseq", "source"]
df = df[df.source.isin(author2pmc)].copy()
df["PMC"] = df.source.apply(lambda x: author2pmc[x])
df["mirseq"] = df.mirseq.apply(lambda x: x.replace("U", "T"))
df

In [ ]:
goldDoc2Mirna = defaultdict(set)
for ri, row in df.iterrows():
    goldDoc2Mirna[row["PMC"]].add(row["mirseq"])

for x in goldDoc2Mirna:
    print(x, len(goldDoc2Mirna[x]))

In [ ]:
for x in goldDoc2Mirna:
    if x in origin2tax2mirna:
        for taxID in origin2tax2mirna[x]:
            print(x, taxID, len(goldDoc2Mirna[x]), len(goldDoc2Mirna[x].intersection(origin2tax2mirnaseq[x][taxID])), len(origin2tax2mirna[x][taxID]), len(origin2tax2mirnaseq[x][taxID]))


In [ ]:
for docID in sorted(origin2tax2mirna):
    for taxID in origin2tax2mirna[docID]:
        print(docID, taxID, len(origin2tax2mirna[docID][taxID]), len(origin2tax2mirnaseq[docID][taxID]))

In [ ]:
[x for x in origin2tax2mirnaseq["PMC7278893"]]

In [ ]:
goldExtra = goldDoc2Mirna["PMC8719879"].difference(origin2tax2mirnaseq["PMC8719879"]["2697049"])
goldExtra

In [ ]:
goldMissing = origin2tax2mirnaseq["PMC8719879"]["2697049"].difference(goldDoc2Mirna["PMC8719879"])
goldMissing

In [ ]:
len("GATTTGATTACGTCTATAATCC")

In [ ]:
rnaForm = "AATTGTGTTAATTGTGATACAT".replace("T", "U")
print(rnaForm)
print("".join(reversed(rnaForm)))

In [ ]:
targetMirna = None

for docid in origin2tax2mirna:
    for taxid in origin2tax2mirna[docid]:
        for x in origin2tax2mirna[docid][taxid]:

            if x.mirna_seq in goldMissing or x.mirna_seq in goldExtra:
                print(x, x.matched_taxids)
        
                